In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)y
import random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install torch transformers transformer_lens 
!pip install -i https://pypi.org/simple/ bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.9.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.0 which is incompatible.
ydata-profiling 4.9.0 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.0 MB/s eta 0:00:0000:0100:01


In [ ]:
HF_token ="<token>"

In [4]:
use_8bit = True

bnb_config = BitsAndBytesConfig(
    load_in_8bit=use_8bit,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

# Load the Llama 3 8B model and tokenizer
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_token)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
llama_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                   quantization_config=bnb_config,
                                                   device_map="auto",
                                                   token=HF_token)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [6]:
# use_4bit = True
# bnb_4bit_compute_dtype = torch.bfloat16
# bnb_4bit_quant_type = "nf4"
# use_nested_quant = False

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )

# # 
# # Load the Llama 3 8B model and tokenizer
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  
# tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_token)
# tokenizer.pad_token = tokenizer.eos_token

# llama_model = AutoModelForCausalLM.from_pretrained(model_name,
#                                                    quantization_config=bnb_config,
#                                                    device_map="auto",
#                                                    token=HF_token     )

In [65]:
system={"role": "system", 
    "content": 
"""You are an expert in telecommunication systems. Your task is to generate ONE query for a Router Network Placement Bot. 
Your task is only to create one, varied query using the following options.

Mandatory Details:
1. Number of Routers: {2-10}
2. Number of Users: {10-200}
3. Room Dimensions (length, width): {20-100m}

The Four Optional Details are:
1. Transmit Power: {10-30 dBm}
2. Path Loss Parameter K: {0.005-0.05} 
3. Path Loss Parameter alpha: {2-4}
4. Device Name: {Cisco XR-1234, Juniper MX240, Huawei NE40, Nokia 7750, Cisco ASR-900}
5. Room Height: {3-6m}
6. Locations: {Hospital, Warehouse, Office, House, Banquet Hall}QA
7. User Distribution(Mention along both X and Y Axes): {Uniform, Exponential, Gaussian, Bi-Exponential, Spread Evenly, near one corner, near center of edge, near adjacent corners, concentrated in center of room, in opposite corners, along an edge, concentrate in front or back}

Optimisation Objective: REDUCING DISTANCE BETWEEN USER EQUIPMENT/USER(UE) and ROUTER
Write with diverse tones from technical to casual. Feel Free to add Aliases for User Distributions as well

Your generated query must introduce diversity by VARYING the sentence structure and using synonyms or alternative phrases.

IMPORTANT RULES:
1. **Number of Routers, Number of Users and Room Size must always be included.**
2. **You DO NOT need to include optional details.**
3. In some queries, include just the Mandatory Details, with no optional details.
4. Vary the language, structure and value using aliases or synonyms while mentioning USER DISTRIBUTION in each query to maintain diversity.
5. STRICTLY Prefer generating smaller responses using simple sentences, 30-35 words as a human natually would
6. Include None, One, Two, Three, Four, Five, Six, or all 7 Optional Details across different requests

Generate ONE diverse query based on the above rules. Keep your response SHORT and natural. 
User will provide you with query samples take them as reference but do not repeat them.
Start your query WITH A "^" SIGN and with different start phrases from the Sample to avoid repetition. Use synonyms and alternative expressions to enhance diversity.
*Remember: Only ONE query is allowed. Do NOT generate more than one query, DO NOT GENERATE ANYTHING OTHER THAN THE QUERY*"""
        }

messages = [
  system,
  {"role": "user", "content": "I'm trying to configure 6 routers in a 10m by 10m room with 120 users. They’re distributed uniformly across the space, but I need guidance on optimising distances"}]


In [76]:
good_seed_requests = {"I'm trying to configure 6 routers in a 100m by 30m room with 120 users.I need guidance on optimising distances",
                     "Room that's 80m by 40m with 90 users. There are 5 routers available, and I need a layout that reduces distance between Users and Routers. Could you also consider the height being around 4 meters?",
                     "Working with a 73m by 100m banquet hall that has 150 users and 6 Juniper MX240 routers. Users are mostly concentrated in the centre of the room, and I want to focus on keeping my phone near the routers while keeping the transmit power at 20 dBm.",
                     "Can you help place 9 routers in a 78m by 44m hospital with 180 users? I need to be as close as possible to the routers, and I’m using Nokia 7750 devices with a path loss alpha of 2.5. The room height is 6 meters.",
                     "Determine an optimal network setup for a large banquet hall where 6 Huawei NE40 devices serve a densely populated area of 150 users, near an edge of the room, which measures 70m x 100m. minimising distance between users and routers",
                     "I need a placement plan for a 71m by 60m banquet hall with concentrated users along corner of the room, using 6 Cisco XR-1234 routers, aiming to shorten user to router distances and optimal 20 dBm transmit power.",
                     "Optimize Huawei NE40 routers placement in the 7m by 10m banquet hall, while optimising for proximity with a peak load of 150 users concentrated along the room's front at 20 dBm transmit power."}

In [77]:
good_seed_requests

{'Can you help place 9 routers in a 9m by 9m hospital with 180 users? I need to be as close as possible to the routers, and I’m using Nokia 7750 devices with a path loss alpha of 2.5. The room height is 6 meters.',
 'Determine an optimal network setup for a large banquet hall where 6 Huawei NE40 devices serve a densely populated area of 150 users, near an edge of the room, which measures 7m x 10m. minimising distance between users and routers',
 'I need a placement plan for a 7m by 10m banquet hall with concentrated users along corner of the room, using 6 Cisco XR-1234 routers, aiming to shorten user to router distances and optimal 20 dBm transmit power.',
 "I'm trying to configure 6 routers in a 10m by 10m room with 120 users.I need guidance on optimising distances",
 "Optimize Huawei NE40 routers placement in the 7m by 10m banquet hall, while optimising for proximity with a peak load of 150 users concentrated along the room's front at 20 dBm transmit power.",
 "Room that's 8m by 8m w

In [68]:
print(messages[0]["content"])

You are an expert in telecommunication systems. Your task is to generate ONE query for a Router Network Placement Bot. 
Your task is only to create one, varied query using the following options.

Mandatory Details:
1. Number of Routers: {2-10}
2. Number of Users: {10-200}
3. Room Dimensions (length, width): {5-10m}

The Four Optional Details are:
1. Transmit Power: {10-30 dBm}
2. Path Loss Parameter K: {0.005-0.05} 
3. Path Loss Parameter alpha: {2-4}
4. Device Name: {Cisco XR-1234, Juniper MX240, Huawei NE40, Nokia 7750, Cisco ASR-900}
5. Room Height: {3-6m}
6. Locations: {Hospital, Warehouse, Office, House, Banquet Hall}
7. User Distribution(Mention along both X and Y Axes): {Uniform, Exponential, Gaussian, Bi-Exponential, Spread Evenly, near one corner, near center of edge, near adjacent corners, concentrated in center of room, in opposite corners, along an edge, concentrate in front or back}

Optimisation Objective: REDUCING DISTANCE BETWEEN USER EQUIPMENT/USER(UE) and ROUTER
Write

In [69]:
print(messages[1]["content"])

I'm trying to configure 6 routers in a 10m by 10m room with 120 users. They’re distributed uniformly across the space, but I need guidance on optimising distances


In [70]:
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt",padding=True, truncation=True,).to(llama_model.device)
type(inputs)
inputs.shape

torch.Size([1, 613])

In [72]:
with torch.no_grad():
    output = llama_model.generate(inputs, max_length=1000, num_return_sequences=1, temperature=1.25)

modified_prompt = tokenizer.decode(output[0], skip_special_tokens=True)
modified_prompt.split("^")[-1].strip()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"I'd like to simulate a 10m x 10m room with 120 users, evenly dispersed in all areas, placing 6 routers to minimize distances between users and routers."

In [61]:
modified_prompt

'system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are an expert in telecommunication systems. Your task is to generate ONE query for a Router Network Placement Bot. \nYour task is only to create one, varied query using the following options.\n\nMandatory Details:\n1. Number of Routers: {2-10}\n2. Number of Users: {10-200}\n3. Room Dimensions (length, width): {5-10m}\n\nThe Four Optional Details are:\n1. Transmit Power: {10-30 dBm}\n2. Path Loss Parameter K: {0.005-0.05} \n3. Path Loss Parameter alpha: {2-4}\n4. Device Name: {Cisco XR-1234, Juniper MX240, Huawei NE40, Nokia 7750, Cisco ASR-900}\n5. Room Height: {3-6m}\n6. Locations: {Hospital, Warehouse, Office, House, Banquet Hall}\n7. User Distribution(Mention Same or Different Distribution along X and Y Axes): {Uniform, Exponential, Gaussian, Bi-Exponential, Spread Evenly, near one corner, near center of edge, near adjacent corners, concentrated in center of room, in opposite corners, along an edge}\n\n

In [78]:
user_requests= good_seed_requests

In [79]:
good_requests={"Develop a floor plan strategy for 5 RN devices, accommodating a user spread similar to that found in office spaces (uni-exponential) in a rectangular room that's 5m by 10m, with routers positioned at 1.8m height. Ensure the network reaches 90% coverage.",
              "I need to deploy 10 Router Numbers across a rectangular room measuring 6m by 4m with a specific device name like X100 to cater to Uniform-Uniform distribution, can you guide on this.",
              "We require an analysis of suitable locations for placement of 8 Y25 routers in a spacious room that measures 10m x 20m, necessitating a coverage of at least 85% for a Uni-Uni user distribution.",}

In [ ]:
def sample_requests(requests, n=8):
    return random.sample(list(requests), min(n, len(requests)))

def generate_modification(prompt):
    message=[system,
            {"role":"user","content":prompt}]
    input_ids = tokenizer.apply_chat_template(message, return_tensors="pt",padding=True, truncation=True,).to(llama_model.device)
    
    with torch.no_grad():
        output = llama_model.generate(input_ids, max_length=input_ids.shape[1]+150, num_return_sequences=1, temperature=1.25);
    
    modified_prompt = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(modified_prompt)
    return modified_prompt.split("^")[-1].strip()

def main():
    global user_requests
    
    # Sample 8 requests
    while(len(user_requests)<20):
        sampled_requests = sample_requests(user_requests,n=5)

        context = "SAMPLE REQUESTS FOR REFERENCE ARE AS FOLLOWS: \n" 
        count=1
        for request in sampled_requests:
                context+=str(count)+". "+request+"\n"
                count+=1
        print("Context: \n",context)
#         print("----------------")
        for i in range(5):
#             print("context: \n",context)
            modified_request = generate_modification(context)

            print() 
            user_requests.add(modified_request)
            context+=str(count)+". "+modified_request+"\n"
            count+=1

            print(f"Modified request {len(user_requests)+1}: {modified_request}")
            print("----------------")

        print()
    print(f"Updated set of user requests (total: {len(user_requests)}):")
    for request in user_requests:
        print(f"- {request}")

if __name__ == "__main__":
    main()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: 
 SAMPLE REQUESTS FOR REFERENCE ARE AS FOLLOWS: 
1. I need a placement plan for a 7m by 10m banquet hall with concentrated users along corner of the room, using 6 Cisco XR-1234 routers, aiming to shorten user to router distances and optimal 20 dBm transmit power.
2. Room that's 8m by 8m with 90 users. There are 5 routers available, and I need a layout that reduces distance between Users and Routers. Could you also consider the height being around 4 meters?
3. Can you help place 9 routers in a 9m by 9m hospital with 180 users? I need to be as close as possible to the routers, and I’m using Nokia 7750 devices with a path loss alpha of 2.5. The room height is 6 meters.
4. Determine an optimal network setup for a large banquet hall where 6 Huawei NE40 devices serve a densely populated area of 150 users, near an edge of the room, which measures 7m x 10m. minimising distance between users and routers
5. Working with a 7m by 10m banquet hall that has 150 users and 6 Juniper MX240 rou

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Modified request 9: Optimise router placement for a 9.5m by 6m office area that holds 175 users densely distributed near an edge, using 8 Cisco XR-1234 devices to significantly shorten UE to router distances, and set room height to approximately 5 meters.
----------------


In [18]:
user_requests

{'" SIGN and with different start phrases to avoid repetition. Use synonyms and alternative expressions to enhance diversity.\n*Remember: Only ONE query is allowed. Do NOT generate more than one query.*user\n\nSAMPLE REQUESTS FOR REFERENCE ARE AS FOLLOWS: \n1. Design a high-performance router network for an Office room (7m x 8m) with a concentration of users along the edges, optimizing the signal to achieve the lowest possible transmission latency, using 9 Cisco ASR-900 devices.\n\n(Please provide feedback to generate another diverse query.)\n2. To plan a robust router network for a spacious Office (10m x 8m), using {9} Juniper MX240 devices and minimizing router-user distance, distribute users along the center and adjacent corners.\n\n(Note: I\'ve followed your rules and guidelines, varying the sentence structure, synonyms, and language to ensure diversity in my response. Let me know if you need any adjustments or another query!)\n3. I need a placement plan for a 7m by 10m banquet hal

In [19]:
with open('output_Q8.txt', 'w') as f:
    f.write(str(user_requests))